# Hồi quy đa biến (gradient descent)

Trong notebook đầu tiên, chúng ta đã khám phá hồi quy đa biến sử dụng mô hình mặc định của sklearn. Giờ chúng ta sẽ sử dụng riêng numpy để giải quyết các trọng số hồi quy với gradient descent.

Trong notebook này, chúng ta sẽ đề cập tới các trọng số của hồi quy đa biến qua gradient descent. Chúng ta sẽ:
* Thêm một cột không đổi của 1 vào DataFrame để tính intercept.
* Xuất DataFrame hoặc cột (Series) thành một mảng Numpy.
* Viết hàm predict_output() sử dụng Numpy.
* Viết một hàm numpy để tính đạo hàm của các trọng số hồi quy với một đặc trưng duy nhất.
* Viết hàm gradient descent tính các trọng số hồi quy biết vectơ trọng số ban đầu, kích thước bước và dung sai.
* Sử dụng hàm gradient descent để ước tính các trọng số hồi quy cho nhiều đặc trưng.

# Load thư viện

Đảm bảo đã có các thư viện theo yêu cầu.

In [1]:
import sklearn, pandas
import numpy as np

## Load dữ liệu doanh số bán nhà

Tập dữ liệu doanh số bán nhà ở quận King, Seatle, WA. Nghe quen chứ?

In [2]:
full_data = pandas.read_csv("house_data/kc_house_data.csv", index_col=0)
full_data.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,,,,,,,,,,,,,,,,,,,,
7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


Nếu muốn thực hiện bất kỳ "feature engineering" nào như tạo các đặc trưng mới hoặc điều chỉnh đặc trưng sẵn có, chúng ta có thể sửa DataFrame của pandas như trong lab trước (*Lab 2*). Tuy nhiên, với notebook này, chúng ta sẽ làm việc với các đặc trưng có sẵn.

## Chuyển thành mảng Numpy

Để hiểu chi tiết về việc triển khai các thuật toán, cần làm việc với một thư viện cho phép thoa tác trực tiếp với ma trận (và được tối ưu hóa). Numpy là một giải pháp Python để làm việc với ma trận (hoặc bất kỳ "mảng" đa chiều nào).

Giá trị dự đoán cho các trọng số và đặc trưng chỉ là tích vô hướng giữa đặc trưng và vectơ trọng số. Tương tự, nếu chúng ta đặt tất cả các đặc trưng thành từng hàng trong một ma trận thì có thể tính giá trị dự đoán cho *tất cả* các quan sát bằng cách nhân "ma trận đặc trưng" với "vectơ trọng số".

Trước tiên, chúng ta cần lấy DataFrame đã sắp xếp và trích xuất dữ liệu bên dưới dưới thành một mảng numpy 2D (còn được gọi là ma trận). Để làm điều này, chúng ta có thể sử dụng thuộc tính .values của Panda để chuyển đổi dataframe thành một ma trận numpy. 

In [3]:
import numpy as np # điều này cho phép gọi numpy as np

Bây giờ chúng ta sẽ viết một hàm nhận DataFrame, một list tên các đặc trưng (chẳng hạn: ['sqft_living', 'bedrooms']) và một đặc trưng mục tiêu (ví dụ: 'price') và trả về 2 điều sau:
* Một ma trận numpy có các cột là các đặc trưng mong muốn cộng với một cột không đổi (đây là cách chúng ta tạo 'intercept').
* Một mảng numpy chứa các giá trị của đầu ra.

Với những điều này, hãy hoàn thành hàm sau (nếu có dòng trống, hãy viết một dòng code thực hiện những gì chú giải ở trên chỉ ra).

In [4]:
def get_numpy_data(data, features_title, output_title):
    if('constant' not in data):
        data['constant'] = 1 # đây là cách thêm cột constant. Chỉ thực hiện khi cần 
    # thêm cột 'constant' vào trước list các đặc trưng để chúng ta có thể trích xuất cùng với những thứ khác:
    features_title = ['constant'] + features_title # đây là cách kết hợp 2 list
    # chia dữ liệu thành sub-DataFrame chứa các đặc trưng đã chỉ định (gồm constant)
    # gọi nó là features_columns.
    features_columns = data[features_title]

    # dòng tiếp theo sẽ trích xuất ma trận numpy từ biến features_columns:
    feature_matrix = features_columns.values
    # truy xuất dữ liệu được liên kết với đầu ra trong pandas Series
    # gọi nó là output_column
    output_column = data[output_title]

    # tiếp theo sẽ chuyển đổi Series đã nhắc thành một mảng numpy
    output_array = output_column.values
    return(feature_matrix, output_array)

**Suy nghĩ: Lab trước không hề chỉ định bất kỳ cột Constant nào. Chúng ta có mắc lỗi ở đâu không?**

>> Trả lời: Chúng ta không mắc lỗi, Lab trước sử dụng thư viện sklearn. Trong LinearRegression đã có thuộc tính Constant.

Chúng ta sẽ sử dụng đặc trưng 'sqft_living' và một hằng số làm đặc trưng và giá làm đầu ra để kiểm tra:

In [5]:
(example_features, example_output) = get_numpy_data(full_data, ['sqft_living'], 'price') # [] quanh 'sqft_living' tạo một list
print(example_features[0,:]) # điều này truy cập hàng đầu tiên của dữ liệu, ':' chỉ 'all columns'
print(example_output[0]) # và đầu ra tương ứng

[   1 1180]
221900.0


## Dự đoán đầu ra với các trọng số hồi quy đã cho

Giả sử chúng ta có trọng số [1.0, 1.0] và đặc trưng [1.0, 1180.0], chúng ta muốn tính kết quả dự đoán $1.0*1.0 + 1.0*1180.0 = 1181.0$ (đây là tích vô hướng giữa 2 mảng). Nếu chúng là mảng numpy, chúng ta có thể dùng `np.dot` để tính:

In [6]:
my_weights = np.array([1., 1.]) # trọng số mẫu
my_features = example_features[0,] # chúng ta sẽ dùng điểm dữ liệu đầu tiên
predicted_value = np.dot(my_features, my_weights)
print(predicted_value)

1181.0


`np.dot` cũng hoạt động khi xử lý ma trận và vectơ. Dự đoán từ các quan sát là tích vô hướng ĐÚNG (như trọng số ở bên phải) giữa các *ma trận* đặc trưng và *vectơ* trọng số. Hãy hoàn thành hàm `predict_output` sau để tính các dự đoán cho toàn bộ ma trận đặc trưng với ma trận và các trọng số đã cho:

In [7]:
def predict_output(feature_matrix, weights):
    # giả sử ma trận feature_matrix chứa các đặc trưng ở dạng các cột và trọng số là mảng numpy tương ứng
    # tạo vectơ dự đoán sử dụng np.dot()
    predictions = np.dot(feature_matrix, weights)

    return(predictions)

Chạy cell say và quan sát kết quả nếu muốn kiểm tra code:

In [8]:
test_predictions = predict_output(example_features, my_weights)
print(test_predictions[0]) # nên là 1181.0
print(test_predictions[1]) # nên là 2571.0

1181.0
2571.0


# Tính đạo hàm

Bây giờ chúng ta sẽ chuyển sang tính đạo hàm của hàm chi phí hồi quy. Hàm chi phí là tổng các điểm dữ liệu của hiệu bình phương giữa kết quả quan sát và kết quả dự đoán.

Vì đạo hàm của một tổng là tổng các đạo hàm nên chúng ta có thể tính đạo hàm cho từng điểm dữ liệu và rồi tính tổng các điểm dữ liệu. Chúng ta có thể viết hiệu bình phương giữa kết quả quan sát và kết quả dự đoán như sau: 

$(w_0 * const + w_1 *feature_1 + ... + w_i  * feature_i + ... +  w_k * feature_k - output)^2$

Chúng ta có k đặc trưng và một hằng số. Như vậy theo quy tắc dây chuyền, đạo hàm với trọng số $w_i$ là:

$2 * (w_0 * const + w_1 *feature_1 + ... + w_i  * feature_i + ... +  w_k * feature_k - output) * feature_i$

Phần tử bên trong ngoặc là sai số (hiệu giữa dự đoán và kết quả). Như vậy, chúng ta có thể viết lại thành:

$2 * error * feature_i$

Đạo hàm cho trọng số của đặc trưng $ i $ là tổng (các điểm dữ liệu) của 2 nhân tích của error và đặc trưng đó. Trong trường hợp đặc trưng là constant thì là gấp đôi tổng sai số!


2 lần tổng của hai vectơ chỉ là hai nhân tích của hai vectơ. Do đó, đạo hàm cho trọng số của $ feature_i $ bằng hai lần tích vô hướng giữa các giá trị của $ feature_i $ và các sai số hiện tại.

Hãy hoàn thành hàm đạo hàm sau đây để tính đạo hàm của trọng số cho giá trị của đặc trưng (trên tất cả các điểm dữ liệu) và sai số (trên tất cả các điểm dữ liệu). 

In [9]:
def feature_derivative(errors, feature):
    # Giả sử sai số và đặc trưng đều là mảng numpy có cùng độ dài (số điểm dữ liệu)
    # tính 2 lần tích vô hướng của các vectơ đó làm 'đạo hàm' và trả về giá trị
    derivative = 2 * (errors * feature).sum()

    return(derivative)

Để kiểm tra đạo hàm, chạy cell sau: 

In [10]:
(example_features, example_output) = get_numpy_data(full_data, ['sqft_living'], 'price') 
my_weights = np.array([0., 0.]) # this makes all the predictions 0 điều này làm cho tất cả dự đoán là 0
test_predictions = predict_output(example_features, my_weights) 
# cũng giống như SFrames, 2 mảng numpy có thể trừ với '-':
errors = test_predictions - example_output # sai số dự đoán trong trường hợp này chỉ là example_output
feature = example_features[:,0] # tính đạo hàm với 'constant', ":" chỉ tất cả các hàng
print("feature: ", feature)
derivative = feature_derivative(errors, feature)
print(derivative)
print(-np.sum(example_output)*2) # nên giống với đạo hàm

feature:  [1 1 1 ... 1 1 1]
-23345850016.0
-23345850016.0


# Gradient Descent

Bây giờ chúng ta sẽ viết một hàm thực hiện gradient descent. Tiền đề cơ bản khá đơn giản. Với một điểm bắt đầu, chúng ta cập nhật các trọng số hiện tại bằng cách di chuyển theo hướng gradient âm. Gradient có hướng *tăng* nên gradient âm có hướng *giảm* và chúng ta đang cố gắng *giảm thiểu* hàm chi phí.

Lượng mà chúng ta di chuyển theo *hướng* gradient âm được gọi là 'kích thước bước'. Chúng ta dừng lại khi chúng ta 'đủ gần' với mức tối ưu. Điều này được xác định bằng cách yêu cầu độ lớn (chiều dài) của vectơ gradient phải nhỏ hơn một 'dung sai' cố định.

Hãy hoàn thành hàm gradient descent sau bằng cách sử dụng hàm đạo hàm ở trên. Với mỗi bước trong gradient descent, chúng ta cập nhật trọng số cho từng đặc trưng trước khi tính tiêu chí dừng.

In [11]:
from math import sqrt # độ lớn/chiều dài của một vectơ [g[0], g[1], g[2]] là căn bậc hai của (g[0]^2 + g[1]^2 + g[2]^2)

In [12]:
def regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance):
    converged = False 
    weights = np.array(initial_weights) # đảm bảo đây là mảng numpy
    while not converged:
        # tính các dự đoán dựa trên feature_matrix và các trọng số sử dụng hàm predict_output() function
        predictions = predict_output(feature_matrix, weights)
        # tính sai số dưới dạng dự đoán - đầu ra
        residuals = predictions - output

        gradient_sum_squares = 0 # khởi tạo gradient_sum_squares
        # khi chưa đạt tới dung sai, hãy cập nhất trọng số cho từng đặc trưng
        for i in range(len(weights)): # lặp qua từng trọng số
            # feature_matrix[:, i] là cột đặc trưng liên kết với weights[i]
            # tính đạo hàm cho weight[i]:
            derivative = feature_derivative(residuals, feature_matrix[:, i])
            
            # cộng bình phương giá trị của đạo hàm vào tổng bình phương gradient (để đánh giá hội tụ)
            gradient_sum_squares += derivative**2
            
            # trọng số hiện tại trừ stepsize nhân với đạo hàm
            weights[i] -= step_size * derivative
            
        # tính căn bậc hai của tổng bình phương gradient để lấy độ lớn của gradient:
        gradient_magnitude = sqrt(gradient_sum_squares)
        if gradient_magnitude < tolerance:
            converged = True
    return(weights)

Một số điều cần lưu ý trước khi chạy gradient descent: Vì gradient là tổng của tất cả các điểm dữ liệu và liên quan đến tích của sai số và đặc trưng nên bản thân gradient sẽ rất lớn do các đặc trưng (squarefeet) và đầu ra (giá) lớn. Vì vậy, mặc dù chúng ta dự kiến "dung sai" nhỏ, nhưng chỉ nhỏ tương đối với kích thước các đặc trưng.

Tương tự, kích thước bước sẽ nhỏ hơn nhiều so với dự kiến nhưng điều này là do gradient có các giá trị lớn. 

# Chạy Gradient Descent như Hồi quy đơn giản

Trước tiên, hãy chia thành tập huấn luyện và tập kiểm tra.

In [13]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(full_data, train_size=0.8, test_size=0.2, random_state=0)

Mặc dù gradient descent được thiết kế cho hồi quy đa biến vì hằng số bây giờ là một đặc trưng, chúng ta có thể sử dụng hàm gradient descent để ước tính các tham số trong hồi quy đơn giản trên squarefeet. Các cell sau thiết lập feature_matrix, đầu ra, trọng số ban đầu và kích thước bước cho mô hình đầu tiên:

In [14]:
# hãy kiểm tra gradient descent
simple_features = ['sqft_living']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(train_data, simple_features, my_output)
initial_weights = np.array([-47000., 1.])
step_size = 7e-12
tolerance = 2.5e7

Tiếp theo, chúng ta sẽ chạy gradient descent với các tham số trên.

In [15]:
updated_weights = regression_gradient_descent(simple_feature_matrix, output, initial_weights, step_size, tolerance)

In [16]:
updated_weights

array([-46999.88720259,    283.46383063])

Các trọng số này so với những trọng số đạt được ở tuần 1 thế nào (không dự kiến chúng giống hệt nhau)? 

>> Answer: Trọng số xấp xỉ với trọng số phần 1, không thể giống hệt nhau vì trong phần 1 là tính toán trực tiếp
>> Còn dùng gradient decent là một phương pháp dùng để lấy xấp xỉ 

**Quiz: Giá trị của trọng số cho sqft_living -- phần tử thứ hai của ‘simple_weights’ là bao nhiêu (làm trong tới chữ số thập phân thứ nhất)?**

>> Answer: 283.46

In [17]:
# Có thể in ra tất cả trọng số nếu muốn
updated_weights[1].round(decimals=2)

283.46

Hãy sử dụng các trọng số mới ước tính và `predict_output` để tính các dự đoán trên dữ liệu KIỂM TRA (cần tạo một mảng numpy của feature_matrix và kiểm tra đầu ra trước tiên):

In [18]:
(test_simple_feature_matrix, test_output) = get_numpy_data(test_data, simple_features, my_output)

Bây giờ có thể tính các dự đoán sử dụng test_simple_feature_matrix và các trọng số ở trên.

In [19]:
# sử dụng predict_output()
predicted = predict_output(test_simple_feature_matrix, updated_weights)

**Quiz: Giá dự đoán cho ngôi nhà đầu tiên trong tập dữ liệu KIỂM TRA cho mô hình 1 là bao nhiêu (làm tròn thành đô la)?**

In [20]:
# index đầu tiên trong ngôn ngữ lập trình là gì?
predicted[0]

358353.39059137006

Giờ chúng ta đã có các dự đoán trên dữ liệu kiểm tra, tính RSS trên tập kiểm tra. Lưu giá trị này để so sánh sau. Nhắc lại rằng RSS là tổng các sai số bình phương (hiệu giữa dự đoán và kết quả).

In [21]:
# trừ, bình phương, cộng lại và in
residuals = predicted - test_output
residuals_square = residuals**2
rss = residuals_square.sum()
print("Residual sum of squares:", rss)

Residual sum of squares: 267729995270518.6


# Chạy hồi quy đa biến 

Bây giờ chúng ta sẽ dùng nhiều hơn một đặc trưng. Sử dụng code sau để tạo ra các trọng số cho mô hình thứ hai với các tham số sau:

In [22]:
model_features = ['sqft_living', 'sqft_living15'] # sqft_living15 là diện tích trung bình cho 15 hàng xóm gần nhất. 
my_output = 'price'
(feature_matrix, output) = get_numpy_data(train_data, model_features, my_output)
initial_weights = np.array([-100000., 1., 1.])
step_size = 4e-12
tolerance = 1e9

Sử dụng các tham số trên để ước tính trọng số mô hình. Ghi lại các giá trị này cho quiz.

In [23]:
weights = regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance)
weights

array([-9.99999757e+04,  2.47055837e+02,  6.47974873e+01])

Sử dụng các trọng số mới ước tính và hàm `predict_output` để tính các dự đoán trên dữ liệu KIỂM TRA. Đừng quên tạo một mảng numpy cho các đặc trưng từ tập kiểm tra đầu tiên!

In [24]:
# ba cái thứ hai vẫn chưa được truyền.
(test_feature_matrix, test_output) = get_numpy_data(test_data, model_features, my_output)
predicted = predict_output(test_feature_matrix, weights)
predicted

array([ 345950.27809068, 1327844.15463587,  349068.81133431, ...,
        306625.22382674,  204684.35573932,  410143.52646188])

**Quiz: Giá dự đoán cho ngôi nhà thứ nhất trong tập dữ liệu KIỂM TRA cho mô hình 2 là bao nhiêu (làm tròn thành đô la)?**

In [25]:
# một lần nữa, index đầu tiên
predicted[0]


345950.2780906761

Giá trị thực cho ngôi nhà thứ nhất trong tập dữ liệu kiểm tra là bao nhiêu?

In [26]:
# tìm nó trong test_output
test_output[0]

297000.0

**Quiz: Ước tính nào gần với giá thực cho ngôi nhà thứ nhất trong tập KIỂM TRA: mô hình 1 hay 2?**

Sử dụng các dự đoán và kết quả để tính RSS cho mô hình 2 trên dữ liệu KIỂM TRA.

In [27]:
# trừ,... Đợi chút. Copy, paste và sửa 
residuals = predicted - test_output
residuals_square = residuals**2
rss = residuals_square.sum()
rss

262684098596668.75

**Quiz: Mô hình nào (1 hay 2) có RSS thấp nhất trong tất cả dữ liệu KIỂM TRA?**

In [28]:
rss_model1 = 267729995270518.6
rss_model2 = 262684098596668.75

In [29]:
rss_model1 > rss_model2

True